# The Enron Email Dataset

## 1. Downloading data

More info about dataset: [link](https://www.kaggle.com/datasets/wcukierski/enron-email-dataset)

Please provide your Kaggle credentials to download this dataset. Learn more: [link](http://bit.ly/kaggle-creds)

In [1]:
import os, sys, email
import pandas as pd
import numpy as np

import opendatasets as od

od.download_kaggle_dataset("https://www.kaggle.com/datasets/wcukierski/enron-email-dataset", data_dir='.')

In [1]:
data = pd.read_csv('enron-email-dataset/emails.csv')
data.head()

FileNotFoundError: [Errno 2] No such file or directory: 'enron-email-dataset/emails.csv'

In [3]:
print(data.message[0])

Message-ID: <18782981.1075855378110.JavaMail.evans@thyme>
Date: Mon, 14 May 2001 16:39:00 -0700 (PDT)
From: phillip.allen@enron.com
To: tim.belden@enron.com
Subject: 
Mime-Version: 1.0
Content-Type: text/plain; charset=us-ascii
Content-Transfer-Encoding: 7bit
X-From: Phillip K Allen
X-To: Tim Belden <Tim Belden/Enron@EnronXGate>
X-cc: 
X-bcc: 
X-Folder: \Phillip_Allen_Jan2002_1\Allen, Phillip K.\'Sent Mail
X-Origin: Allen-P
X-FileName: pallen (Non-Privileged).pst

Here is our forecast

 


## 2. Parse data

Let's define some useful function for email parsing

In [15]:
def get_text_from_email(msg: email.message.Message) -> str:
    """To get the content from email objects"""
    parts = []
    for part in msg.walk():
        if part.get_content_type() == 'text/plain':
            parts.append( part.get_payload() )
    return ''.join(parts)

def split_email_addresses(line: str) -> frozenset:
    """To separate multiple email addresses"""
    if line:
        addresses = line.split(',')
        addresses = list(map(lambda x: x.strip(), addresses))
    else:
        addresses = list()
    return addresses

In [16]:
df = pd.DataFrame()
df['Date'] = data['message'].apply(lambda msg: pd.to_datetime(email.message_from_string(msg)['Date'], errors='ignore'))
df['From'] = data['message'].apply(lambda msg: split_email_addresses(email.message_from_string(msg)['From']))
df['To'] = data['message'].apply(lambda msg: split_email_addresses(email.message_from_string(msg)['To']))
df['Subject'] = data['message'].apply(lambda msg: email.message_from_string(msg)['Subject'])
df['SubjectType'] = df['Subject'].apply(lambda x: 'replied' if 're:' == x[:3].lower() else
                                                  'forwarded' if 'fw:' == x[:3].lower() else
                                                  'empty' if len(x) == 0 else
                                                  'ordinary')
df['Content'] = data['message'].apply(lambda msg: get_text_from_email(email.message_from_string(msg)))
df = df.drop_duplicates().sort_values(by='Date', ignore_index=True)
df['From'] = df['From'].apply(list)
df['To'] = df['To'].apply(list)
df.head()

,Date,From,To,Subject,SubjectType,Content
0,1979-12-31 16:00:00-08:00,[debra.perlingiere@enron.com],[hissongm@dteenergy.com],Re: DTE Energy Trading--Spec Prov to GISB; GISB,replied,"Marcia,\n\nThank you for your e-mail. Concern..."
1,1979-12-31 16:00:00-08:00,[mark.haedicke@enron.com],[],Creating Worldwide Electronic Trading Form,ordinary,I believe the start-up of click trading is an ...
2,1979-12-31 16:00:00-08:00,[vince.kaminski@enron.com],[rick.buy@enron.com],Re: Exmar Purchase Decision,replied,"Rick,\n\nIt would be difficult to use option a..."
3,1979-12-31 16:00:00-08:00,[janette.elbertson@enron.com],[],Enron Wholesale Services Legal Department,ordinary,As a follow-up to the recent Enron Corp. memo...
4,1979-12-31 16:00:00-08:00,[janette.elbertson@enron.com],"[raymond.bowen@enron.com, david.delainey@enron...",Chinese Wall Resource Group,ordinary,You have been designated as a member of the =...


Load to csv

In [15]:
df.to_csv('./enron-email-dataset/messages.csv', sep='|')

Description of `messages`:
- `Date` - date of sending message
- `From` - who send message
- `To` - who get message
- `Subject` - email's subject
- `SubjectType` - one of [`empty`, `replied`, `forwarded`, `ordinary`]
- `Content` - content of message

## 3. PostgresSQL

In [5]:
import psycopg2
from sqlalchemy import create_engine

In [17]:
engine = create_engine('postgresql+psycopg2://:@localhost/dismissal_prediction_service')

In [ ]:
with open("../db/scripts/create_table_messages.sql") as f:
    table_creation_query = f.read()

engine

### 3.1. First and last send message date per employee

In [14]:
query = """
WITH corporate_e
SELECT "Email",
        MAX("Date") AS "LastDate",
        MIN("Date") AS "FirstDate"
FROM mail.messages
WHERE "Email" LIKE "
"""

,Date,From
0,2044-01-04 22:48:58,"('cramer@cadvision.com',)"
1,2044-01-04 16:59:46,"('cramer@cadvision.com',)"
2,2044-01-02 23:46:00,"('cramer@cadvision.com',)"
3,2043-12-28 19:34:12,"('cramer@cadvision.com',)"
4,2024-05-26 10:49:57,"('pse6yl706@aloha.net',)"
...,...,...
255469,1980-01-01 00:00:00,"('peter.keavey@enron.com',)"
255470,1980-01-01 00:00:00,"('richard.sanders@enron.com',)"
255471,1980-01-01 00:00:00,"('harry.arora@enron.com',)"
255472,1980-01-01 00:00:00,"('steven.kean@enron.com',)"
